## 🏗️ Strategic Planning: The Plan-and-Execute Pattern

While the ReAct pattern is great for exploration, it can sometimes lose its way in highly complex tasks. The Plan-and-Execute pattern addresses this by separating reasoning into two distinct phases:

1. **Phase 1 (The Architect):** Analyzes the request and builds a complete, step-by-step roadmap.
2. **Phase 2 (The Worker):** Executes the roadmap systematically using tools.

### 🎯 Learning Objectives

* **Strategic Reasoning:** Learn to prompt the model to build an upfront plan without using tools.
* **Predictability:** Understand how a pre-defined plan makes agent behavior easier to audit and debug.
* **Complex Orchestration:** Observe the agent managing multi-step inventory audits and trend analysis.
* **Separation of Concerns:** See the difference between a "Planner" prompt and an "Executor" prompt.

In [ ]:
%pip install openai python-dotenv

In [ ]:
# Setup with z.ai
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import sqlite3
from datetime import datetime
from typing import Dict, Any

# Load environment variables
load_dotenv()

# Initialize OpenAI client (points to z.ai)
client = OpenAI(
    base_url=os.environ.get("ZAI_BASE_URL"),
    api_key=os.environ.get("ZAI_API_KEY")
)

MODEL = os.environ.get("ZAI_MODEL", "gpt-4o")

print("✅ Plan & Execute Agent - Using LLM via z.ai")
print(f"🔗 Endpoint: {os.environ.get('ZAI_BASE_URL')}")
print(f"🤖 Model: {MODEL}")

In [ ]:
# Initialize Databases (reset to clean state)
import importlib
import db_helper
importlib.reload(db_helper)
from db_helper import reset_inventory_database, reset_database

reset_database()
reset_inventory_database()

In [ ]:
# Phase 1 - Planning Function
def create_plan(task_description: str) -> str:
    """
    The Architect Phase: Creates a detailed roadmap WITHOUT executing tools.
    This phase is pure reasoning.
    """
    
    planning_prompt = f"""You are a senior inventory strategist. Always respond in English.
                        Given a task, create a logical, step-by-step plan.
                        
                        Available tools for the execution phase:
                        - check_stock(product_id): Current stock levels.
                        - search_inventory(category, low_stock_only): Filtered search. Category can be 'Electronics', 'Office Supplies', 'Furniture', or omit for all.
                        - get_sales_trend(product_id): Sales forecast data.
                        - create_purchase_order(product_id, quantity, reason): Reorder items.
                        
                        Your plan must:
                        1. Be a numbered list of specific actions.
                        2. State which tool to use for each step.
                        3. Explain the logic behind the step.
                        
                        Output ONLY the numbered list in English.
                        
                        Task: {task_description}"""
    
    print("🤔 PHASE 1: STRATEGIC PLANNING")
    print("="*80)
    
    response = client.chat.completions.create(
        model=MODEL,
        max_tokens=2048,
        messages=[{"role": "user", "content": planning_prompt}]
    )
    
    plan = response.choices[0].message.content
    print(f"📋 THE ROADMAP:\n{plan}\n" + "-"*80)
    return plan

In [ ]:
# Phase 2 - Execution Function
import importlib
import re
import inventory_tools
importlib.reload(inventory_tools)
from inventory_tools import INVENTORY_TOOLS, check_stock, search_inventory, get_sales_trend, create_purchase_order

def clean_malformed_tool_calls(text: str) -> str:
    """Remove malformed tool call syntax that GLM sometimes outputs as text"""
    if not text:
        return text
    # Remove XML-like tool call patterns
    text = re.sub(r'<arg_key>.*?</arg_key>\s*', '', text, flags=re.DOTALL)
    text = re.sub(r'<arg_value>.*?</arg_value>\s*', '', text, flags=re.DOTALL)
    text = re.sub(r'</tool_call>\s*', '', text)
    text = re.sub(r'\b(search_inventory|check_stock|get_sales_trend|create_purchase_order)\s*$', '', text, flags=re.MULTILINE)
    return text.strip()

def execute_plan(task_description: str, plan: str, max_iterations: int = 15):
    """
    The Worker Phase: Follows the roadmap step-by-step.
    """
    print("⚙️ PHASE 2: EXECUTION")
    print("="*80)
    
    system_prompt = """You are an inventory manager. Follow the provided plan exactly. 
    Use tools for data retrieval only. For the final summary report, just write the report as text - do NOT use any tools.
    Always respond in English."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Execute this plan:\n{plan}\n\nOriginal Task: {task_description}"}
    ]
    
    for iteration in range(max_iterations):
        print(f"📍 STEP {iteration + 1}")
        
        response = client.chat.completions.create(
            model=MODEL,
            max_tokens=2048,
            messages=messages,
            tools=INVENTORY_TOOLS
        )
        
        assistant_message = response.choices[0].message
        
        # Display Progress (text content)
        if assistant_message.content:
            cleaned_content = clean_malformed_tool_calls(assistant_message.content)
            if cleaned_content:
                print(f"💭 {cleaned_content}")
        
        if response.choices[0].finish_reason != "tool_calls":
            print("="*80 + "\n✅ TASK COMPLETE")
            final_text = assistant_message.content or ""
            return clean_malformed_tool_calls(final_text)
        
        messages.append(assistant_message)
        
        # Tool Execution Loop
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_input = json.loads(tool_call.function.arguments)
            
            print(f"🔧 CALLING: {tool_name}")
            print(f"   Input: {tool_input}")
            
            # Route to the appropriate tool function
            if tool_name == "check_stock":
                result = check_stock(tool_input["product_id"])
            elif tool_name == "search_inventory":
                # Handle various ways the model might indicate "all categories"
                category = tool_input.get("category")
                if category in ("", "all", "All", "ALL", "None", "none", "null", None):
                    category = None
                result = search_inventory(
                    category,
                    tool_input.get("low_stock_only", False)
                )
            elif tool_name == "get_sales_trend":
                result = get_sales_trend(tool_input["product_id"])
            elif tool_name == "create_purchase_order":
                result = create_purchase_order(
                    tool_input["product_id"],
                    tool_input["quantity"],
                    tool_input["reason"]
                )
            else:
                result = {"error": f"Unknown tool: {tool_name}"}
            
            print(f"📊 DATA: {json.dumps(result, indent=2)}\n")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })

In [ ]:
# Complete Planning Agent
def planning_agent(task_description: str, max_iterations: int = 15):
    """
    Complete Planning Agent: Plan first, then execute
    
    This combines both phases:
    1. Create a comprehensive plan
    2. Execute the plan step by step
    
    Args:
        task_description: What to accomplish
        max_iterations: Max execution steps
        
    Returns:
        Final results
    """
    
    print("="*80)
    print(f"🎯 TASK: {task_description}")
    print("="*80)
    print()
    
    # Phase 1: Create the plan
    plan = create_plan(task_description)
    
    # Phase 2: Execute the plan
    result = execute_plan(task_description, plan, max_iterations)
    
    return result

### Test 1: Targeted Inspection

**Success:** The agent should identify the correct category and check stock without unnecessary steps.

In [ ]:
# Test 1 - Simple Inventory Check
planning_agent("Show me all products that are currently low on stock");

### Test 2: Analytical Reasoning

**Success:** The agent must find the sales trend and provide a recommendation based on the forecast, not just current numbers.

In [ ]:
# Test 2 - Sales Analysis
planning_agent("Analyze the sales trend for the USB-C Cable (PROD002) and recommend whether we should reorder");

### Test 3: The Full Audit (Multi-Entity)

**Success:** The agent must create multiple purchase orders across different products after checking trends for each.

#### **Horizontal Execution:** The agent finds all items that are low on stock and performs a standard check for each. 

In [ ]:
# Test 3 - Complex Multi-Step Task
planning_agent("""
                Perform a comprehensive inventory audit:
                1. Find all products that are low on stock
                2. For each low-stock product, check its sales trend
                3. Create purchase orders for products that will run out in the next 7 days
                4. Provide a summary report of all actions taken
                """);

### Test 4: The Strategic Inventory Audit

This test is the ultimate demonstration of the Plan-and-Execute architecture. The agent must act as a manager, performing a top-to-bottom audit of an entire category.

#### **Vertical Analysis:** The agent focuses on one category (Electronics) and must perform multiple analytical steps per item. It doesn't just check stock, it has to calculate `Average Daily Sales` to figure out how much to order, not just if it should order.

In [ ]:
# Test 4 - Category Analysis
planning_agent("""
                Analyze the Electronics category:
                1. Find all electronics products
                2. Check which ones are low on stock
                3. For low stock items, analyze their sales trends
                4. Recommend reorder quantities based on average daily sales
                5. Create purchase orders for critical items
                """);